In [7]:
import json
import os
import pandas as pd
import isodate
import requests
import numpy as np
import cv2
from collections import Counter
from sklearn.cluster import KMeans
import time
from tqdm import tqdm
import random

In [242]:
merge_df=pd.read_csv("final_video_datas.csv", encoding="utf-8-sig")

In [244]:
merge_df.info()
merge_df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7308 entries, 0 to 7307
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title_x             7308 non-null   object 
 1   video_id            7308 non-null   object 
 2   published_date      7308 non-null   object 
 3   thumbnail_url       7308 non-null   object 
 4   view_count          7308 non-null   float64
 5   like_count          7175 non-null   float64
 6   comment_count       7121 non-null   float64
 7   duration            7308 non-null   float64
 8   channel_id          7308 non-null   object 
 9   subscriber_count    7308 non-null   float64
 10  brightness          7308 non-null   float64
 11  contrast            7308 non-null   float64
 12  dominant_colors     7308 non-null   object 
 13  thumbnail_size      7308 non-null   object 
 14  object_labels       7308 non-null   object 
 15  text_positions      7308 non-null   object 
 16  person

,title_x,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,duration,channel_id,subscriber_count,...,word_count,emoji_count,has_emoji,special_char_count,is_clickbait,has_question_mark,has_exclamation,top_noun_1,top_noun_2,top_noun_3
count,7308,7308,7308,7308,7.308000e+03,7175.000000,7121.000000,7308.000000,7308,7.308000e+03,...,7308.000000,7308.000000,7308.000000,7308.000000,7308.000000,7308,7308,7283,7128,6846
unique,7289,7308,1073,7308,NaN,NaN,NaN,NaN,2453,NaN,...,NaN,NaN,NaN,NaN,NaN,2,2,2789,3106,3158
top,[12뉴스] 오늘의 주요뉴스 / SBS,t-zwVkTOZA8,2025-03-28,https://i.ytimg.com/vi/t-zwVkTOZA8/maxresdefau...,NaN,NaN,NaN,NaN,"(' YTN', '@ytnnews24')",NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,테슬라,버라이어티,하바나
freq,11,1,221,1,NaN,NaN,NaN,NaN,201,NaN,...,NaN,NaN,NaN,NaN,NaN,5861,5578,284,101,67
mean,NaN,NaN,NaN,NaN,6.481823e+05,9409.151916,782.451341,925.517378,NaN,1.517002e+06,...,19.960865,0.351259,0.197729,4.297072,0.057745,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1.460531e+06,29279.895063,3528.673610,709.398507,NaN,3.908397e+06,...,9.051113,0.860403,0.398314,3.278131,0.233276,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,4.320000e+02,0.000000,0.000000,60.000000,NaN,5.280000e+02,...,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,3.151825e+04,466.500000,62.000000,355.750000,NaN,1.190000e+05,...,14.000000,0.000000,0.000000,2.000000,0.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,1.465250e+05,2099.000000,205.000000,780.000000,NaN,4.610000e+05,...,19.000000,0.000000,0.000000,4.000000,0.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,6.055885e+05,7457.500000,664.000000,1301.000000,NaN,1.880000e+06,...,25.000000,0.000000,0.000000,6.000000,0.000000,NaN,NaN,NaN,NaN,NaN


In [19]:
cat_cols = ['has_emoji', 'is_clickbait', 'has_question_mark', 'has_exclamation']
for col in cat_cols:
    print(merge_df[col].value_counts())

0    5863
1    1445
Name: has_emoji, dtype: int64
0    6886
1     422
Name: is_clickbait, dtype: int64
False    5861
True     1447
Name: has_question_mark, dtype: int64
False    5578
True     1730
Name: has_exclamation, dtype: int64


In [246]:
def fill_by_group_median(df, group_col, target_col):
    return df[target_col].fillna(
        df.groupby(group_col)[target_col].transform('median')
    )

# 1. 결측 여부 피처
merge_df['like_missing'] = merge_df['like_count'].isnull().astype(int)
merge_df['comment_missing'] = merge_df['comment_count'].isnull().astype(int)

# 2. 구독자수 구간화 → subscriber_group 컬럼 생성
merge_df['subscriber_group'] = pd.qcut(
    merge_df['subscriber_count'], 
    q=5,
    labels=False,
    duplicates='drop'
)

# 3. subscriber_group 기준으로 중앙값 채우기
merge_df['like_count'] = fill_by_group_median(merge_df, 'subscriber_group', 'like_count')
merge_df['comment_count'] = fill_by_group_median(merge_df, 'subscriber_group', 'comment_count')

In [248]:
merge_df['published_date'] = pd.to_datetime(merge_df['published_date'])
merge_df['pub_year'] = merge_df['published_date'].dt.year
merge_df['pub_month'] = merge_df['published_date'].dt.month
merge_df['pub_weekday'] = merge_df['published_date'].dt.weekday

In [71]:
# 트리기반모델 쓰는 경우 생략 가능
# from sklearn.preprocessing import MinMaxScaler
# # 정규화
# scaler = MinMaxScaler()
# scaled_cols = ['like_count', 'comment_count', 'subscriber_count', 'duration', 'brightness', 'contrast']
# merge_df[scaled_cols] = scaler.fit_transform(merge_df[scaled_cols])

### 주요단어 벡터화

#### TF-IDF

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# 1. 주요 명사 결합
merge_df['nouns_combined'] = merge_df[['top_noun_1', 'top_noun_2', 'top_noun_3']].fillna('').apply(
    lambda row: ' '.join([str(noun).strip() for noun in row if str(noun).strip() != '']), axis=1
)

# 2. TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(merge_df['nouns_combined'])

# 3. 전체 단어별 평균 TF-IDF를 계산해 상위 n개 선택
tfidf_means = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
top_n = 30
top_n_indices = np.argsort(tfidf_means)[::-1][:top_n]
top_n_features = [tfidf.get_feature_names_out()[i] for i in top_n_indices]

# 4. 상위 단어만 선택한 DataFrame 만들기
top_n_matrix = tfidf_matrix[:, top_n_indices]
top_n_df = pd.DataFrame(top_n_matrix.toarray(), columns=[f"tfidf_{word}" for word in top_n_features])

# 5. 병합
merge_df = pd.concat([merge_df.reset_index(drop=True), top_n_df.reset_index(drop=True)], axis=1)

'tfidf_테슬라', 'tfidf_뉴스', 'tfidf_전기차', 'tfidf_닌텐도',
       'tfidf_아이폰', 'tfidf_스위치', 'tfidf_게임', 'tfidf_경제', 'tfidf_한국',
       'tfidf_인공', 'tfidf_여행', 'tfidf_버라이어티', 'tfidf_미국', 'tfidf_호텔',
       'tfidf_일상', 'tfidf_먹방', 'tfidf_운동', 'tfidf_중국', 'tfidf_지능', 'tfidf_갱생',
       'tfidf_하바나', 'tfidf_메이크업', 'tfidf_트럼프', 'tfidf_머스크', 'tfidf_지금',
       'tfidf_이유', 'tfidf_토크쇼', 'tfidf_일본', 'tfidf_사람', 'tfidf_패션'

#### 차원 축소

In [81]:
from sklearn.decomposition import TruncatedSVD

# 기존 TF-IDF 전체 벡터 그대로 사용
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(merge_df['nouns_combined'])

# SVD로 차원 축소
svd = TruncatedSVD(n_components=20, random_state=42)
svd_matrix = svd.fit_transform(tfidf_matrix)

# SVD 결과를 DataFrame으로 변환
svd_df = pd.DataFrame(svd_matrix, columns=[f"tfidf_svd_{i+1}" for i in range(svd_matrix.shape[1])])

# 병합
merge_df = pd.concat([merge_df.reset_index(drop=True), svd_df.reset_index(drop=True)], axis=1)

 'tfidf_svd_1', 'tfidf_svd_2', 'tfidf_svd_3', 'tfidf_svd_4',
       'tfidf_svd_5', 'tfidf_svd_6', 'tfidf_svd_7', 'tfidf_svd_8',
       'tfidf_svd_9', 'tfidf_svd_10', 'tfidf_svd_11', 'tfidf_svd_12',
       'tfidf_svd_13', 'tfidf_svd_14', 'tfidf_svd_15', 'tfidf_svd_16',
       'tfidf_svd_17', 'tfidf_svd_18', 'tfidf_svd_19', 'tfidf_svd_20'

## 객체 & 텍스트 전처리

In [250]:
import ast
# 문자열을 실제 리스트로 변환 (object_labels 칼럼에만 적용)
merge_df['object_labels'] = merge_df['object_labels'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
merge_df['dominant_colors'] = merge_df['dominant_colors'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

def safe_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return []
    return val if isinstance(val, list) else []

# 적용
merge_df['person_positions'] = merge_df['person_positions'].apply(safe_literal_eval)
merge_df['text_positions'] = merge_df['text_positions'].apply(safe_literal_eval)

In [45]:
color_li=[]
for i in merge_df['dominant_colors']:
    for j in i:
        color_li.append(j[0])

set(color_li)

{'aliceblue',
 'antiquewhite',
 'aquamarine',
 'azure',
 'beige',
 'bisque',
 'black',
 'blanchedalmond',
 'blue',
 'blueviolet',
 'brown',
 'burlywood',
 'cadetblue',
 'chartreuse',
 'chocolate',
 'coral',
 'cornflowerblue',
 'cornsilk',
 'crimson',
 'cyan',
 'darkblue',
 'darkcyan',
 'darkgoldenrod',
 'darkgray',
 'darkgreen',
 'darkgrey',
 'darkkhaki',
 'darkmagenta',
 'darkolivegreen',
 'darkorange',
 'darkorchid',
 'darkred',
 'darksalmon',
 'darkseagreen',
 'darkslateblue',
 'darkslategrey',
 'darkturquoise',
 'darkviolet',
 'deeppink',
 'deepskyblue',
 'dimgrey',
 'dodgerblue',
 'firebrick',
 'floralwhite',
 'forestgreen',
 'gainsboro',
 'ghostwhite',
 'gold',
 'goldenrod',
 'green',
 'greenyellow',
 'grey',
 'honeydew',
 'hotpink',
 'indianred',
 'indigo',
 'ivory',
 'khaki',
 'lavender',
 'lavenderblush',
 'lawngreen',
 'lemonchiffon',
 'lightblue',
 'lightcoral',
 'lightcyan',
 'lightgoldenrodyellow',
 'lightgray',
 'lightgreen',
 'lightgrey',
 'lightpink',
 'lightsalmon',
 '

In [252]:
from collections import Counter, defaultdict
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
import ast

def preprocess_for_model(df, top_tfidf_n=50):
    df = df.copy()

    color_groups = {
        'red': {'red', 'crimson', 'firebrick', 'darkred', 'salmon', 'indianred', 'tomato', 'orangered'},
        'blue': {'blue', 'navy', 'dodgerblue', 'deepskyblue', 'royalblue', 'skyblue', 'slateblue', 'mediumblue'},
        'green': {'green', 'lime', 'forestgreen', 'seagreen', 'springgreen', 'mediumseagreen', 'darkgreen', 'lawngreen'},
        'yellow': {'yellow', 'gold', 'khaki', 'lemonchiffon', 'lightyellow', 'palegoldenrod'},
        'purple': {'purple', 'magenta', 'violet', 'orchid', 'mediumorchid', 'mediumpurple', 'darkviolet'},
        'brown': {'brown', 'sienna', 'chocolate', 'peru', 'saddlebrown', 'tan', 'burlywood'},
        'grey': {'grey', 'gray', 'lightgrey', 'darkgrey', 'dimgrey', 'slategrey', 'gainsboro'},
        'white': {'white', 'snow', 'ivory', 'ghostwhite', 'whitesmoke', 'floralwhite', 'seashell'},
        'pink': {'pink', 'lightpink', 'hotpink', 'lavenderblush'},
        'other': set()
    }

    def map_color_to_group(color):
        for group, colors in color_groups.items():
            if color.lower() in colors:
                return group
        return 'other'

    for col in color_groups.keys():
        df[f'color_{col}'] = 0.0

    def count_color_groups(color_list):
        counter = defaultdict(float)
        for color, prob in color_list:
            group = map_color_to_group(color)
            counter[group] += prob
        return counter

    for idx, row in df.iterrows():
        if isinstance(row['dominant_colors'], list):
            counter = count_color_groups(row['dominant_colors'])
            for group in counter:
                df.at[idx, f'color_{group}'] = counter[group]

    df['person_count'] = df['object_labels'].apply(lambda x: x.count('person') if isinstance(x, list) else 0)

    all_objects = []
    for x in df['object_labels']:
        if isinstance(x, list):
            all_objects.extend([obj for obj in x if obj != 'person'])

    top_objects = [obj for obj, _ in Counter(all_objects).most_common(10)]

    for obj in top_objects:
        df[f'obj_{obj}'] = df['object_labels'].apply(lambda x: int(obj in x) if isinstance(x, list) else 0)

    df['has_text'] = df['text_positions'].apply(lambda x: int(x != ['텍스트 없음']))
    df['has_person_pos'] = df['person_positions'].apply(lambda x: int(x != ['사람 없음']))

    # tfidf = TfidfVectorizer(max_features=top_tfidf_n)
    # tfidf_matrix = tfidf.fit_transform(df['title_x'].fillna(''))
    # tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'tfidf_{w}' for w in tfidf.get_feature_names_out()])
    # df = pd.concat([df.reset_index(drop=True), tfidf_df], axis=1)

    return df

In [254]:
model_df = preprocess_for_model(merge_df)

In [255]:
# 사람 위치
model_df['person_left'] = model_df['person_positions'].apply(lambda x: sum('left' in p for p in x))
model_df['person_middle'] = model_df['person_positions'].apply(lambda x: sum('middle' in p for p in x))
model_df['person_right'] = model_df['person_positions'].apply(lambda x: sum('right' in p for p in x))
model_df['person_small'] = model_df['person_positions'].apply(lambda x: sum('s' in p for p in x))
model_df['person_medium'] = model_df['person_positions'].apply(lambda x: sum('m' in p for p in x))
model_df['person_large'] = model_df['person_positions'].apply(lambda x: sum('l' in p for p in x))

# 텍스트 위치
model_df['text_left'] = model_df['text_positions'].apply(lambda x: sum('left' in p for p in x))
model_df['text_middle'] = model_df['text_positions'].apply(lambda x: sum('middle' in p for p in x))
model_df['text_right'] = model_df['text_positions'].apply(lambda x: sum('right' in p for p in x))
model_df['text_small'] = model_df['text_positions'].apply(lambda x: sum('s' in p for p in x))
model_df['text_medium'] = model_df['text_positions'].apply(lambda x: sum('m' in p for p in x))
model_df['text_large'] = model_df['text_positions'].apply(lambda x: sum('l' in p for p in x))

In [260]:
model_columns=['view_count',
       'like_count', 'comment_count', 'duration',
       'subscriber_count', 'brightness', 'contrast', 'title_length', 'word_count', 'emoji_count', 'has_emoji',
       'special_char_count', 'is_clickbait', 'has_question_mark',
       'has_exclamation', 'pub_year',
       'pub_month', 'pub_weekday',
       'color_red', 'color_blue', 'color_green', 'color_yellow',
       'color_purple', 'color_brown', 'color_grey', 'color_white',
       'color_pink', 'color_other', 
        'person_count',
        'obj_bowl', 'obj_없음', 'obj_car', 'obj_tie', 'obj_chair', 'obj_cake', 'obj_cup',
       'obj_cell phone', 'obj_bottle', 'obj_teddy bear', 
        'has_text', 'has_person_pos', 'person_left', 'person_middle', 'person_right',
       'person_small', 'person_medium', 'person_large', 'text_left',
       'text_middle', 'text_right', 'text_small', 'text_medium', 'text_large']

model_df=model_df[model_columns]

In [262]:
model_df.to_csv('model_datas.csv', index=False, encoding="utf-8-sig",)

In [264]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기
df = pd.read_csv("model_datas.csv")

# 타겟과 피처 분리
X = df.drop(columns=["view_count"])
y = df["view_count"]

# 학습/검증 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 선택 및 학습 - RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_val)

# 평가
rmse = mean_squared_error(y_val, y_pred, squared=False)
r2 = r2_score(y_val, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

RMSE: 655146.30
R²: 0.7567
